# An introduction to dask-kubernetes and intake-esm

Now that TRACMIP is embedded in Pangeo's [climate catalog](https://catalog.pangeo.io/climate), it is easier to aggregate through and merge the 7067 zarr sources that comprise the collection.
Additionally, with dask-kubernetes, it should be much easier to perform computation on this data, as well as checking the performance of these computations using Dask's distributed dashboard.

To initialize a Dask client, use the lab extension to start a new cluster and instantiate a `Client` with the scheduler address:

In [7]:
from dask.distributed import Client

client = Client("tcp://10.32.2.19:37469")
client

Client Scheduler: tcp://10.32.2.19:37469 Dashboard: /user/0000-0003-2292-0572/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


From here, the number of workers can be scaled manually or adaptively.

To access TRACMIP's ESM collection:

In [8]:
from intake import open_catalog

cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/climate.yaml")
col = cat.tracmip()
col

pangeo-tracmip-ESM Collection with 7067 entries:
	> 3 frequency(s)

	> 11 experiment(s)

	> 14 model(s)

	> 47 variable(s)

	> 10 version(s)

	> 7067 source(s)

From here, `col.df` can be used to access the list of zarr sources like a Pandas `DataFrame`:

In [9]:
col.df.head()

,frequency,experiment,model,variable,version,source
0,A3hr,aqua4xCO2,AM21,hur,v20190116,gs://cmip6/tracmip/A3hr/aqua4xCO2/AM21/hur/v20...
1,A3hr,aqua4xCO2,AM21,hus,v20190116,gs://cmip6/tracmip/A3hr/aqua4xCO2/AM21/hus/v20...
2,A3hr,aqua4xCO2,AM21,ta,v20190116,gs://cmip6/tracmip/A3hr/aqua4xCO2/AM21/ta/v201...
3,A3hr,aqua4xCO2,AM21,ua,v20190116,gs://cmip6/tracmip/A3hr/aqua4xCO2/AM21/ua/v201...
4,A3hr,aqua4xCO2,AM21,va,v20190116,gs://cmip6/tracmip/A3hr/aqua4xCO2/AM21/va/v201...


To find the columns and their unique values:

In [10]:
col.df.columns

Index(['frequency', 'experiment', 'model', 'variable', 'version', 'source'], dtype='object')

In [11]:
col.unique(["experiment", "model", "variable"])

{'experiment': {'count': 11,
  'values': ['aqua4xCO2',
   'aquaControl',
   'land4xCO2',
   'landControl',
   'landOrbit',
   'aquaAbs07',
   'aquaAbs15',
   'aquaAbs20',
   'landAbs07',
   'landAbs15',
   'landAbs20']},
 'model': {'count': 14,
  'values': ['AM21',
   'CAM4',
   'CAM5Nor',
   'CNRM-AM5',
   'ECHAM61',
   'ECHAM63',
   'GISS-ModelE2',
   'MIROC5',
   'MPAS',
   'MetUM-CTL',
   'MetUM-ENT',
   'CAM3',
   'LMDZ5A',
   'CALTECH']},
 'variable': {'count': 47,
  'values': ['hur',
   'hus',
   'ta',
   'ua',
   'va',
   'wap',
   'zg',
   'clivi',
   'clt',
   'clwvi',
   'hfls',
   'hfss',
   'huss',
   'pr',
   'prc',
   'prsn',
   'prw',
   'ps',
   'psl',
   'rlds',
   'rldscs',
   'rlus',
   'rlut',
   'rlutcs',
   'rsds',
   'rsdscs',
   'rsdt',
   'rsus',
   'rsuscs',
   'rsut',
   'rsutcs',
   'rtmt',
   'sfcWind',
   'tas',
   'tasmax',
   'tasmin',
   'tauu',
   'tauv',
   'ts',
   'cl',
   'cli',
   'clw',
   'evspsbl',
   'uas',
   'vas',
   'cct',
   'hurs']}}

To make a query of the collection:

In [12]:
new_col = col.search(frequency="Amon",
                     experiment=["aquaControl", "landControl"],
                     model=["CAM3", "CAM4", "CAM5Nor"],
                     variable=["pr", "cl", "huss"])

To load the collection into lazily using Dask:

In [13]:
ds_dict = new_col.to_dataset_dict(zarr_kwargs={"consolidated" : True})

Progress: |███████████████████████████████████████████████████████████████████████████████| 100.0% 

--> The keys in the returned dictionary of datasets are constructed as follows:
	'model.experiment.frequency'
             
--> There are 6 group(s)


In [14]:
ds_dict.keys()

dict_keys(['CAM3.landControl.Amon', 'CAM3.aquaControl.Amon', 'CAM5Nor.landControl.Amon', 'CAM4.landControl.Amon', 'CAM4.aquaControl.Amon', 'CAM5Nor.aquaControl.Amon'])

In [15]:
ds_dict["CAM3.landControl.Amon"]

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 64, lon: 128, plev: 17, time: 480)
Coordinates:
  * lat        (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 3e+03 2e+03 1e+03
  * time       (time) object 0046-01-16 12:00:00 ... 0085-12-16 12:00:00
Dimensions without coordinates: bnds
Data variables:
    cl         (time, plev, lat, lon) float32 dask.array<chunksize=(240, 17, 64, 128), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(64, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(128, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(480, 2), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(480, 64, 128), meta=np.ndarray>
Attributes:
    parent_experiment_rip:  r1i1p1
    parent_experiment_id:   aquaControlTRACMIP
    initialization_method:  1
    tracking_id:            5a5055f9-1d3a-4248-b0da-dbf9116d5ffe\n41b272bd-29...
    branch_time:            0.0
    product:                output
    project_id:             TRACMIP
    modeling_realm:         atmos
    institute_id:           UW-MADISON
    comment:                idealized land control of TRACMIP; for TRACMIP se...
    cmor_version:           2.9.1
    contact:                rddixon@wisc.edu; ross.dixon@meteo.fr
    Conventions:            CF-1.4
    physics_version:        1
    source:                 CAM3, T42, 26 levels;
    institution:            University of Wisconsin Madison; Centre National ...
    experiment_id:          landControlTRACMIP
    experiment:             idealized continent control of TRACMIP
    forcing:                CTRL
    title:                  CAM3 model output prepared for TRACMIP idealized ...
    realization:            1
    parent_experiment:      aqua planet control of TRACMIP
    references:             CAM3: Collins et al., 2006, The formulation and a...
    model_id:               CAM3
    frequency:              mon
    table_id:               Table Amon (11 September 2017) 8c633597e288c86e41...
    history:                N/A 2018-12-29T20:37:11Z CMOR rewrote data to com...